In [2]:
import pandas as pd

df = pd.read_csv("../Datasets/cpu/AMDfullspecs_adjusted.csv")
original_df = df
# df

In [3]:
# df.describe()

In [4]:
# df.info()

In [5]:
# df.columns

## 1 finding processor not having gpu


In [6]:
set1 = df[
    [
        "model",
        "graphicsModel",
        "launchDate",
        "platform",
        "L1Cache",
        "L2Cache",
        "L3Cache",
    ]
]
set1[set1["graphicsModel"].isna()].shape

(269, 7)

In [7]:
set1[set1["graphicsModel"].isna()][["model", "launchDate", "platform"]]

,model,launchDate,platform
37,AMD EPYC™ 72F3,2021.0,Server
38,AMD EPYC™ 7313,2021.0,Server
39,AMD EPYC™ 7313P,2021.0,Server
40,AMD EPYC™ 7343,2021.0,Server
41,AMD EPYC™ 73F3,2021.0,Server
...,...,...,...
577,TL-56,NaN,Laptop
578,TL-60,NaN,Laptop
579,TL-64,NaN,Laptop
580,X1150,NaN,Server


In [9]:
# cpu not having gpu and is for laptop
laptop_without_gpu = set1[set1["platform"].str.contains("Laptop", case=False, na=False)]
laptop_without_gpu

,model,graphicsModel,launchDate,platform,L1Cache,L2Cache,L3Cache
0,AMD Ryzen™ 5 6600H,AMD Radeon™ 660M,2022.0,Laptop,384.0,3.0,16.0
1,AMD Ryzen™ 5 6600HS​,AMD Radeon™ 660M,2022.0,Laptop,384.0,3.0,16.0
2,AMD Ryzen™ 5 6600U,AMD Radeon™ 660M,2022.0,Laptop,384.0,3.0,16.0
3,AMD Ryzen™ 5 PRO 6650H,AMD Radeon™ 660M,2022.0,Laptop,384.0,3.0,16.0
4,AMD Ryzen™ 5 PRO 6650HS,AMD Radeon™ 660M,2022.0,Laptop,384.0,3.0,16.0
...,...,...,...,...,...,...,...
576,TL-52,NaN,NaN,Laptop,NaN,2.0,NaN
577,TL-56,NaN,NaN,Laptop,NaN,NaN,NaN
578,TL-60,NaN,NaN,Laptop,NaN,1.0,NaN
579,TL-64,NaN,NaN,Laptop,NaN,1.0,NaN


In [8]:
cpu_for_server = set1[set1["platform"].str.contains("Server")]
# cpu_for_server

In [9]:
cpu_for_notebook = set1[set1["platform"].str.contains("Notebook")]
cpu_for_notebook

,model,graphicsModel,launchDate,platform,L1Cache,L2Cache,L3Cache


## 2 CPU grouped by family


In [10]:
cpu_family = df.groupby("family")["model"].apply(list)
# cpu_family

In [11]:
cpu_family_count = df.groupby("family")["model"].count()
# print(cpu_family_count)

In [16]:
# top 5 cpu of family having most head count

highest_freq_family_model = (
    df.groupby("family")["model"].count().idxmax()
)  # this will give model name
top_5_highest_freq = df[df["family"] == highest_freq_family_model].head(5)
# top_5_highest_freq

## 3 Launch date is after 2023

In [17]:
launch_date_2020 = set1[set1["launchDate"] > 2020].sort_values(
    "launchDate", ascending=False
)[["model", "launchDate", "platform"]]
latest_cpu_for_laptop = launch_date_2020[
    launch_date_2020["platform"].str.contains("Laptop")
]
# latest_cpu_for_laptop

## 4 Top 5 cpus having higest l1, l2, l3 cache 

In [18]:
l1Cache_list = df[["model", "L1Cache", "platform"]]
h = l1Cache_list["L1Cache"].max()  # int datatype for comparision
# l1Cache_list[l1Cache_list["L1Cache"] == h][["model", "L1Cache", "platform"]]

In [19]:
l2cache_list = df[["L2Cache", "model", "platform"]]
h1 = l2cache_list["L2Cache"].max()
# l2cache_list[l2cache_list["L2Cache"] == h1][["model", "platform", "L2Cache"]]

In [20]:
l3cache_list = df[["model", "L3Cache", "platform"]]
h2 = l3cache_list["L3Cache"].max()
# l3cache_list[l3cache_list["L3Cache"] == h2]

## 6 Filter High-End CPUs

In [135]:
# List all CPUs that: # Have ≥ 8 cores AND ≥ 16 threads # Clock speed (base or boost) ≥ 4 GHz

core_tread_list = df[
    ["model", "numCores", "numThreads", "platform", "baseClock", "launchDate"]
]
core_tread_list[
    (core_tread_list["numCores"] >= 8)
    & (core_tread_list["numThreads"] >= 16)
    & (core_tread_list["baseClock"] >= 4)
]

,model,numCores,numThreads,platform,baseClock,launchDate
103,AMD Ryzen™ Threadripper™ PRO 3945WX,12.0,24.0,Desktop,4.0,2020.0
511,AMD Ryzen™ Threadripper™ PRO 5945WX,12.0,24.0,Desktop,4.1,NaN
512,AMD Ryzen™ Threadripper™ PRO 5955WX,16.0,32.0,Desktop,4.0,NaN


## 7 Extract Year from launchDate

In [150]:
# checking on which year how many cpu launched
cpu_launch_count = df.groupby("launchDate").count()
cpu_launch_count[cpu_launch_count["model"] > 50]  # year when more than 50 cpu launched

,model,numCores,numThreads,baseClock,maxboostClock,allCoreBoostSpeed,L1Cache,L2Cache,L3Cache,processTech,...,graphicsFreq,unlocked,platform,cpuSocket,socketCount,family,line,graphicsModel,thermalSolution,launchYear
launchDate,,,,,,,,,,,,,,,,,,,,,
2020.0,60,60,58,60,60,0,35,60,58,60,...,47,47,60,53,4,60,60,47,4,60
2021.0,56,56,56,56,56,0,4,37,56,37,...,35,21,56,56,19,56,56,35,4,56


## 8 Compute Performance Metrics

In [158]:
# Create a new column: Performance Score = (baseClock + maxboostClock) * numThreads

set2 = df[["model", "baseClock", "maxboostClock", "numThreads", "platform"]]
set2["ps"] = (set2["baseClock"] + set2["maxboostClock"]) * set2["numThreads"]
set2.sort_values("ps", ascending=False)

/tmp/ipykernel_65626/3812531532.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set2["ps"] = (set2["baseClock"] + set2["maxboostClock"]) * set2["numThreads"]


,model,baseClock,maxboostClock,numThreads,platform,ps
515,AMD Ryzen™ Threadripper™ PRO 5995WX,2.70,4.5,128.0,Desktop,921.6
510,AMD Ryzen™ Threadripper™ 3990X,2.90,4.3,128.0,Desktop,921.6
106,AMD Ryzen™ Threadripper™ PRO 3995WX,2.70,4.2,128.0,Desktop,883.2
55,AMD EPYC™ 7763,2.45,3.5,128.0,Server,761.6
482,AMD EPYC™ 7H12,2.60,3.3,128.0,Server,755.2
...,...,...,...,...,...,...
577,TL-56,1.80,NaN,NaN,Laptop,NaN
578,TL-60,2.00,NaN,2.0,Laptop,NaN
579,TL-64,2.20,NaN,2.0,Laptop,NaN
580,X1150,2.00,NaN,4.0,Server,NaN


## 9 Cache Analysis

In [34]:
# converting cache in mb to kb
mb_to_kb = df[["L1Cache", "L2Cache", "L3Cache"]].fillna(0).apply(lambda x: x * 1024.00)
# mb_to_kb

## 10 Top 5 Powerhouse CPUs (OverAll)

In [47]:
# Define "powerhouse" by:
# Threads ≥ 32
# L3 cache ≥ 64 MB
# Boost clock ≥ 4.0 GHz
# Show top 5 CPUs matching these specs sorted by launch date

data_ph = df[["model", "numThreads", "L3Cache", "maxboostClock", "platform", "launchDate"]]
ans = data_ph[(data_ph["numThreads"] >= 32) & (data_ph['L3Cache'] >= 64) & (data_ph['maxboostClock'] >= 4)]
ans.sort_values("launchDate", ascending=False).head(5)

,model,numThreads,L3Cache,maxboostClock,platform,launchDate
41,AMD EPYC™ 73F3,32.0,256.0,4.0,Server,2021.0
43,AMD EPYC™ 7443,48.0,128.0,4.0,Server,2021.0
44,AMD EPYC™ 7443P,48.0,128.0,4.0,Server,2021.0
46,AMD EPYC™ 74F3,48.0,256.0,4.0,Server,2021.0
50,AMD EPYC™ 75F3,64.0,256.0,4.0,Server,2021.0
